In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DataType, StringType, StructType, StructField, TimestampType
from pyspark.sql.functions import from_json, col, to_date, to_timestamp
from delta.tables import *

In [2]:
spark = (
    SparkSession
    .builder
    .appName("incrementa_reservas_delta")
    .config("spark.jars", "/opt/spark/jars/gcs-connector-hadoop3-latest.jar")
    # .config("spark.jars", "./spark/jars/spark-sql-kafka-0-10_2.12-3.1.2.jar")
    #com.google.cloud.bigdataoss:gcs-connector:1.9.4-hadoop3
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0,io.delta:delta-contribs_2.12:1.0.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.delta.logStore.gs.impl", "io.delta.storage.GCSLogStore")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
    .config("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
    .config("fs.gs.auth.service.account.enable", "true")
    .config("fs.gs.auth.service.account.json.keyfile", "/etc/gcp/sa_credentials.json")
    .enableHiveSupport()
    .config("spark.driver.memory", "8g")
    .getOrCreate()
)

21/11/29 18:08:28 WARN Utils: Your hostname, eduney-pc resolves to a loopback address: 127.0.1.1; using 172.24.114.205 instead (on interface eth0)
21/11/29 18:08:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/eduney/.ivy2/cache
The jars for the packages stored in: /home/eduney/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
io.delta#delta-contribs_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ebf28720-f17a-4667-b9a7-8c41054105c0;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/eduney/.local/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found io.delta#delta-contribs_2.12;1.0.0 in central
:: resolution report :: resolve 213ms :: artifacts dl 7ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-contribs_2.12;1.0.0 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-runtime;3.5.2 from central in [default]
	org.antlr#antlr4;4.7 from central in [default]
	org.antlr#antlr4-runtime;4.7 from central in [default]
	org.glassfish#javax.json;1.0

In [3]:
df = (
    spark
    .read
    .format("parquet")
    .load("gs://poc_delta/stage/")
)

In [4]:
df.show(10)

+-----------+-------+-----------+---------+--------------------+------------+--------------------+
|cod_reserva|agencia|dt_retirada|cod_grupo|        dt_geracao_f|dt_devolucao|          dt_geracao|
+-----------+-------+-----------+---------+--------------------+------------+--------------------+
|   19812892|    MCP| 2021-12-11|        B|2021-11-29 16:52:...|  2021-12-16|2021-11-29 16:52:...|
|   61381162|    THE| 2021-12-01|       GX|2021-11-29 16:52:...|  2021-12-06|2021-11-29 16:52:...|
|   99007065|    PMW| 2021-12-04|        C|2021-11-29 16:52:...|  2021-12-09|2021-11-29 16:52:...|
|   75329358|    SDU| 2021-12-12|        B|2021-11-29 16:52:...|  2021-12-17|2021-11-29 16:52:...|
|   21666223|    VCP| 2021-12-07|        B|2021-11-29 16:52:...|  2021-12-12|2021-11-29 16:52:...|
|   42839804|    SSA| 2021-12-06|        C|2021-11-29 16:52:...|  2021-12-11|2021-11-29 16:52:...|
|   60226228|    MCP| 2021-12-06|        B|2021-11-29 16:52:...|  2021-12-11|2021-11-29 16:52:...|
|   708869

In [10]:
spark.sql("""
          select count(cod_reserva)
          from delta.`gs://poc_delta/stage/`
          """).show()

21/11/29 18:22:30 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException


+------------------+
|count(cod_reserva)|
+------------------+
|               882|
+------------------+

